# Metadata
```
Course:   DS 5001 
Module:   02 Lab
Topic:    Text into Data: Importing a Text, or, Clip, Chunk, and Split
Author:   R.C. Alvarado
Purpose:  Demonstrate how to tokenize a raw text and map an OHCO onto the resulting dataframe of tokens.
```

# Set Up

In [1]:
import pandas as pd

In [2]:
data_home = "../data"

In [3]:
text_file = f"{data_home}/gutenberg/pg105.txt"
csv_file = f"{data_home}/output/austen-persuasion.csv" # The file we will create

In [4]:
OHCO = ['chap_num', 'para_num', 'sent_num', 'token_num']

# Import file into a dataframe

In [5]:
LINES = pd.DataFrame(open(text_file, 'r', encoding='utf-8-sig').readlines(), columns=['line_str'])
LINES.index.name = 'line_num'
LINES.line_str = LINES.line_str.str.replace(r'\n+', ' ', regex=True).str.strip()

In [6]:
# LINES.shape

In [7]:
LINES.sample(20)

line_str
line_num                                                   
3687      few fervent ejaculations of gratitude to Heave...
7309      circumstances, could only be exposing them to ...
5184      and home, presiding and blessing in the same s...
8396      rules is very easy.  You may use this eBook fo...
5790      He is always with them; half lives in the fami...
691       circumstances of the Admiral's family, which m...
2720      speech of Louisa's which struck her.  After on...
2362      morning, when he had no companion at home, for...
2119                                            Wentworth."
361       it always does of our conduct.  I have great h...
7972      many, many years of division and estrangement....
5372      know whether the Crofts travelled with four ho...
5306      going on; always the last of my family to be n...
4119      my little shaving glass in one corner, and ano...
4127      him my compliments and Mrs Croft's, and say th...
6121                                                       
4111      wife should have the credit of them, however. ...
6950      woman as you, it was not absolutely hopeless. ...
3994      own sister's intimacy with Mrs Clay.  She was ...
8716      with anyone.  For thirty years, he produced an...

# Extract Title 

In [8]:
title = LINES.loc[0].line_str.replace('The Project Gutenberg EBook of ', '')

In [9]:
print(title)

Persuasion, by Jane Austen


# Clip Cruft

In [10]:
clip_pats = [
    r"\*\*\*\s*START OF (?:THE|THIS) PROJECT", 
    r"\*\*\*\s*END OF (?:THE|THIS) PROJECT"
]

In [11]:
pat_a = LINES.line_str.str.match(clip_pats[0])
pat_b = LINES.line_str.str.match(clip_pats[1])

In [12]:
line_a = LINES.loc[pat_a].index[0] + 1
line_b = LINES.loc[pat_b].index[0] - 1

In [13]:
line_a, line_b

(19, 8372)

In [14]:
LINES = LINES.loc[line_a : line_b]

In [15]:
LINES.head(10)

line_str
line_num                                                   
19                                                         
20                                                         
21                                                         
22                                                         
23        Produced by Sharon Partridge and Martin Ward. ...
24                                            by Al Haines.
25                                                         
26                                                         
27                                                         
28

In [16]:
LINES.tail(10)

line_str
line_num                                                   
8363                                                       
8364                                                       
8365                                                       
8366                                                       
8367                                                       
8368                                                       
8369                                                       
8370                                                       
8371      End of the Project Gutenberg EBook of Persuasi...
8372

# Chunk by chapter

## Find all chapter headers

The regex will depend on the source text. You need to investigate the source text to figure this out.

In [17]:
chap_pat = r"^\s*(?:chapter|letter)\s+\d+"

In [18]:
chap_lines = LINES.line_str.str.match(chap_pat, case=False) # Returns a truth vector

In [19]:
LINES.loc[chap_lines] # Use as filter for dataframe

line_str
line_num            
47         Chapter 1
306        Chapter 2
500        Chapter 3
786        Chapter 4
959        Chapter 5
1297       Chapter 6
1657       Chapter 7
1992       Chapter 8
2346       Chapter 9
2632      Chapter 10
3020      Chapter 11
3314      Chapter 12
3876      Chapter 13
4150      Chapter 14
4409      Chapter 15
4677      Chapter 16
4912      Chapter 17
5248      Chapter 18
5654      Chapter 19
5905      Chapter 20
6265      Chapter 21
6970      Chapter 22
7555      Chapter 23
8208      Chapter 24

## Assign numbers to chapters

In [20]:
LINES.loc[chap_lines, 'chap_num'] = [i+1 for i in range(LINES.loc[chap_lines].shape[0])]

In [21]:
LINES.loc[chap_lines]

line_str  chap_num
line_num                      
47         Chapter 1       1.0
306        Chapter 2       2.0
500        Chapter 3       3.0
786        Chapter 4       4.0
959        Chapter 5       5.0
1297       Chapter 6       6.0
1657       Chapter 7       7.0
1992       Chapter 8       8.0
2346       Chapter 9       9.0
2632      Chapter 10      10.0
3020      Chapter 11      11.0
3314      Chapter 12      12.0
3876      Chapter 13      13.0
4150      Chapter 14      14.0
4409      Chapter 15      15.0
4677      Chapter 16      16.0
4912      Chapter 17      17.0
5248      Chapter 18      18.0
5654      Chapter 19      19.0
5905      Chapter 20      20.0
6265      Chapter 21      21.0
6970      Chapter 22      22.0
7555      Chapter 23      23.0
8208      Chapter 24      24.0

Notice that all lines that are not chapter headers have no chapter number assigned to them.

In [22]:
LINES.sample(10)

line_str  chap_num
line_num                                                             
5505      company the little way our road lies together....       NaN
2136      money in her.  A friend of mine and I had such...       NaN
3114      unwearied contemplation; the woody varieties o...       NaN
5146      old schoolfellow, sick and reduced, seemed to ...       NaN
2577      stout, forward child, of two years old, having...       NaN
4222      exceedingly.  His head is full of some books t...       NaN
6270      Mr Elliot would be most likely to call; for to...       NaN
7467      over, hushed into cold composure, determined s...       NaN
7098      tell her so, but I thought her dress hideous t...       NaN
5596      spirit and gentleness being incompatible with ...       NaN

## Forward-fill chapter numbers to following text lines

`ffill()` will replace null values with the previous non-null value.

In [23]:
LINES.chap_num = LINES.chap_num.ffill()

In [24]:
LINES.sample(10)

line_str  chap_num
line_num                                                             
413                                                               2.0
7022                                                             22.0
5843      when the moment approached which must point hi...      19.0
7070                                                             22.0
5798                             her more than her sister."      19.0
5624                                                             18.0
3305      When the evening was over, Anne could not but ...      11.0
2771                                                             10.0
5147      Elliot.  He thought her a most extraordinary y...      17.0
1928      went away, and he said, 'You were so altered h...       7.0

Notice that the lines taht precede our first chapter have no chapters, which is what we want. We need to decide whether to keep these lines as textual front matter or to dispose of them.

In [25]:
LINES.head(20)

line_str  chap_num
line_num                                                             
19                                                                NaN
20                                                                NaN
21                                                                NaN
22                                                                NaN
23        Produced by Sharon Partridge and Martin Ward. ...       NaN
24                                            by Al Haines.       NaN
25                                                                NaN
26                                                                NaN
27                                                                NaN
28                                                                NaN
29                                                                NaN
30                                                                NaN
31                                                                NaN
32                                                                NaN
33                                                                NaN
34                                                                NaN
35                                               Persuasion       NaN
36                                                                NaN
37                                                                NaN
38                                                       by       NaN

## Clean up

In [26]:
LINES = LINES.dropna(subset=['chap_num']) # Remove everything before Chapter 1
# LINES = LINES.loc[~LINES.chap_num.isna()] # Remove everything before Chapter 1 (alternate method)
LINES = LINES.loc[~chap_lines] # Remove chapter heading lines; their work is done
LINES.chap_num = LINES.chap_num.astype('int') # Convert chap_num from float to int

In [27]:
LINES.sample(10)

line_str  chap_num
line_num                                                             
2741                                                               10
8212      it into their heads to marry, they are pretty ...        24
4766      observation, felt she must submit to.  She cou...        16
4715      If Elizabeth could but have heard this!  Such ...        16
6901      civility, the same hard-hearted indifference t...        21
4989      languor and depression, to hours of occupation...        17
2210      and this is what I do.  I hate to hear of wome...         8
1426      give her the precedence that was her due, when...         6
6555      of a gentleman.  He had always a home with us ...        21
7487                                                               22

## Group lines into chapters

In [28]:
OHCO[:1]

['chap_num']

In [29]:
CHAPS = LINES.groupby(OHCO[:1]).line_str.apply(lambda x: '\n'.join(x)).to_frame('chap_str') # Make big string for each chapter

In [30]:
CHAPS.head(10)

chap_str
chap_num                                                   
1         \n\nSir Walter Elliot, of Kellynch Hall, in So...
2         \n\nMr Shepherd, a civil, cautious lawyer, who...
3         \n\n"I must take leave to observe, Sir Walter,...
4         \n\nHe was not Mr Wentworth, the former curate...
5         \n\nOn the morning appointed for Admiral and M...
6         \n\nAnne had not wanted this visit to Uppercro...
7         \n\nA very few days more, and Captain Wentwort...
8         \n\nFrom this time Captain Wentworth and Anne ...
9         \n\nCaptain Wentworth was come to Kellynch as ...
10        \n\nOther opportunities of making her observat...

In [31]:
CHAPS['chap_str'] = CHAPS.chap_str.str.strip()

In [32]:
CHAPS.head(10)

chap_str
chap_num                                                   
1         Sir Walter Elliot, of Kellynch Hall, in Somers...
2         Mr Shepherd, a civil, cautious lawyer, who, wh...
3         "I must take leave to observe, Sir Walter," sa...
4         He was not Mr Wentworth, the former curate of ...
5         On the morning appointed for Admiral and Mrs C...
6         Anne had not wanted this visit to Uppercross, ...
7         A very few days more, and Captain Wentworth wa...
8         From this time Captain Wentworth and Anne Elli...
9         Captain Wentworth was come to Kellynch as to a...
10        Other opportunities of making her observations...

So, now we have our text grouped by chapters.

# Split chapters into paragraphs 

We use Pandas' convenient `.split()` method with `expand=True`, followed by `.stack()`.
Note that this creates zero-based indexes.

In [33]:
para_pat = r'\n\n+'

In [34]:
# CHAPS['chap_str'].str.split(para_pat, expand=True).head()

In [35]:
PARAS = CHAPS['chap_str'].str.split(para_pat, expand=True).stack()\
    .to_frame('para_str').sort_index()
PARAS.index.names = OHCO[:2]

In [36]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
         1                                 "ELLIOT OF KELLYNCH HALL.
         2         "Walter Elliot, born March 1, 1760, married, J...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...

In [37]:
PARAS['para_str'] = PARAS['para_str'].str.replace(r'\n', ' ', regex=True)
PARAS['para_str'] = PARAS['para_str'].str.strip()
PARAS = PARAS[~PARAS['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

In [38]:
PARAS.head()

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
         1                                 "ELLIOT OF KELLYNCH HALL.
         2         "Walter Elliot, born March 1, 1760, married, J...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...

# Split paragraphs into sentences

In [39]:
# sent_pat = r'[.?!;:"]+'
sent_pat = r'[.?!;:]+'
SENTS = PARAS['para_str'].str.split(sent_pat, expand=True).stack()\
    .to_frame('sent_str')
SENTS.index.names = OHCO[:3]

In [40]:
SENTS = SENTS[~SENTS['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs
SENTS.sent_str = SENTS.sent_str.str.strip() # CRUCIAL TO REMOVE BLANK TOKENS

In [41]:
SENTS.head()

sent_str
chap_num para_num sent_num                                                   
1        0        0         Sir Walter Elliot, of Kellynch Hall, in Somers...
                  1         there he found occupation for an idle hour, an...
                  2         there his faculties were roused into admiratio...
                  3         there any unwelcome sensations, arising from d...
                  4         and there, if every other leaf were powerless,...

# Split sentences into tokens

In [42]:
token_pat = r"[\s',-]+"
TOKENS = SENTS['sent_str'].str.split(token_pat, expand=True).stack()\
    .to_frame('token_str')

In [43]:
TOKENS.index.names = OHCO[:4]

In [44]:
TOKENS

token_str
chap_num para_num sent_num token_num            
1        0        0        0                 Sir
                           1              Walter
                           2              Elliot
                           3                  of
                           4            Kellynch
...                                          ...
24       13       0        6                  of
                           7          Persuasion
                           8                  by
                           9                Jane
                           10             Austen

[85014 rows x 1 columns]

# Extract Vocabulary

In [45]:
TOKENS['term_str'] = TOKENS.token_str.replace(r'[\W_]+', '', regex=True).str.lower()
VOCAB = TOKENS.term_str.value_counts().to_frame('n').reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

In [46]:
VOCAB

term_str     n
term_id                   
0                the  3330
1                 to  2808
2                and  2800
3                 of  2572
4                  a  1595
...              ...   ...
5755            sank     1
5756     infatuation     1
5757          mature     1
5758         jesting     1
5759       welcoming     1

[5760 rows x 2 columns]

# Gathering by Content Object

In [47]:
def gather(ohco_level):
    global TOKENS
    level_name = OHCO[ohco_level-1].split('_')[0]
    df = TOKENS.groupby(OHCO[:ohco_level])\
        .token_str.apply(lambda x: x.str.cat(sep=' '))\
        .to_frame(f"{level_name}_str")
    return df

In [48]:
gather(1)

chap_str
chap_num                                                   
1         Sir Walter Elliot of Kellynch Hall in Somerset...
2         Mr Shepherd a civil cautious lawyer who whatev...
3         "I must take leave to observe Sir Walter " sai...
4         He was not Mr Wentworth the former curate of M...
5         On the morning appointed for Admiral and Mrs C...
6         Anne had not wanted this visit to Uppercross t...
7         A very few days more and Captain Wentworth was...
8         From this time Captain Wentworth and Anne Elli...
9         Captain Wentworth was come to Kellynch as to a...
10        Other opportunities of making her observations...
11        The time now approached for Lady Russell s ret...
12        Anne and Henrietta finding themselves the earl...
13        The remainder of Anne s time at Uppercross com...
14        Though Charles and Mary had remained at Lyme m...
15        Sir Walter had taken a very good house in Camd...
16        There was one point which Anne on returning to...
17        While Sir Walter and Elizabeth were assiduousl...
18        It was the beginning of February and Anne havi...
19        While Admiral Croft was taking this walk with ...
20        Sir Walter his two daughters and Mrs Clay were...
21        Anne recollected with pleasure the next mornin...
22        Anne went home to think over all that she had ...
23        One day only had passed since Anne s conversat...
24        Who can be in doubt of what followed When any ...

In [49]:
gather(2)

para_str
chap_num para_num                                                   
1        0         Sir Walter Elliot of Kellynch Hall in Somerset...
         1                                  "ELLIOT OF KELLYNCH HALL
         2         "Walter Elliot born March 1 1760 married July ...
         3         Precisely such had the paragraph originally st...
         4         Then followed the history and rise of the anci...
...                                                              ...
24       9         Anne satisfied at a very early period of Lady ...
         10        Her recent good offices by Anne had been enoug...
         11        Mrs Smith s enjoyments were not spoiled by thi...
         12                                                    Finis
         13        End of the Project Gutenberg EBook of Persuasi...

[1008 rows x 1 columns]

In [50]:
gather(3)

sent_str
chap_num para_num sent_num                                                   
1        0        0         Sir Walter Elliot of Kellynch Hall in Somerset...
                  1         there he found occupation for an idle hour and...
                  2         there his faculties were roused into admiratio...
                  3         there any unwelcome sensations arising from do...
                  4         and there if every other leaf were powerless h...
...                                                                       ...
24       11       4         Anne was tenderness itself and she had the ful...
                  5         His profession was all that could ever make he...
                  6         She gloried in being a sailor s wife but she m...
         12       0                                                     Finis
         13       0         End of the Project Gutenberg EBook of Persuasi...

[5612 rows x 1 columns]

# Save work to CSV

This is important -- will be used for homework.

In [51]:
TOKENS.to_csv(csv_file)

# Use Library

In [52]:
import sys
sys.path.append("../lib")
from textimporter import TextImporter

In [53]:
pg105 = TextImporter(src_file=text_file, ohco_pats=[('chap', chap_pat, 'm')], clip_pats=clip_pats)

In [54]:
pg105.OHCO

['chap_id', 'para_num', 'sent_num', 'token_num']

In [55]:
pg105.import_source()
pg105.parse_tokens()
pg105.extract_vocab()

Importing  ../data/gutenberg/pg105.txt
Clipping text
Parsing OHCO level 0 chap_id m by milestone ^\s*(?:chapter|letter)\s+\d+
line_str
Parsing OHCO level 1 para_num d by delimitter \n\n
Parsing OHCO level 2 sent_num d by delimitter [.?!;:]+
Parsing OHCO level 3 token_num d by delimitter [\s',-]+


In [56]:
pg105.TOKENS

token_str    term_str
chap_id para_num sent_num token_num                        
1       0        0        0                 Sir         sir
                          1              Walter      walter
                          2              Elliot      elliot
                          3                  of          of
                          4            Kellynch    kellynch
...                                         ...         ...
24      17       0        7                  of          of
                          8          Persuasion  persuasion
                          9                  by          by
                          10               Jane        jane
                          11             Austen      austen

[84904 rows x 2 columns]

In [57]:
pg105.VOCAB

n  n_chars         p             s          i         h
term_str                                                               
the          3330        3  0.039221     25.496697   4.672238  0.183249
to           2808        2  0.033073     30.236467   4.918218  0.162658
and          2800        3  0.032978     30.322857   4.922334  0.162331
of           2572        2  0.030293     33.010886   5.044870  0.152824
a            1595        1  0.018786     53.231348   5.734204  0.107722
...           ...      ...       ...           ...        ...       ...
sank            1        4  0.000012  84904.000000  16.373545  0.000193
infatuation     1       11  0.000012  84904.000000  16.373545  0.000193
mature          1        6  0.000012  84904.000000  16.373545  0.000193
jesting         1        7  0.000012  84904.000000  16.373545  0.000193
welcoming       1        9  0.000012  84904.000000  16.373545  0.000193

[5760 rows x 6 columns]